# Doel van de opdracht

Voor deze opdracht is er gekeken hoe tekstuele data ingezet kan worden voor het maken van een aanbevelingssysteem. Allereerst wordt er gekeken naar de data en het opschonen hiervan, daarna naar geschikte presentatie en als laatst het toepassen van een unsupvervised machine learning.

In [3]:
# Downloads nodig voor het script
import pandas as pd
import re
import nltk
import unicodedata
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\verad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# data inladen en omzetten naar dataframe
df = pd.read_pickle("omdb.pkl")
df.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes_num,Metascore_num,BoxOffice_num,ratings_imdb,ratings_rotten_tomatoes,ratings_metacritic,totalSeasons,Season,Episode,seriesID
imdbID,,,,,,,,,,,,,,,,,,,,,
tt0000010,Leaving the Factory,1895,Not Rated,22 Mar 1895,1 min,"Documentary, Short",Louis Lumière,None,None,A man opens the big gates to the Lumière facto...,...,7738.0,NaN,NaN,6.8,NaN,NaN,NaN,NaN,NaN,NaN
tt0000012,The Arrival of a Train,1896,Not Rated,30 Jun 1896,1 min,"Documentary, Short","Auguste Lumière, Louis Lumière",None,"Madeleine Koehler, Marcel Koehler, Mrs. August...",A group of people are standing in a straight l...,...,13598.0,NaN,NaN,7.4,NaN,NaN,NaN,NaN,NaN,NaN
tt0000014,The Waterer Watered,1895,Not Rated,None,1 min,"Short, Comedy",Louis Lumière,None,"François Clerc, Benoît Duval","A gardener is watering his flowers, when a mis...",...,6220.0,NaN,NaN,7.1,NaN,NaN,NaN,NaN,NaN,NaN
tt0000029,Baby's Meal,1895,None,10 Jul 1896,1 min,"Documentary, Short",Louis Lumière,None,"Auguste Lumière, Mrs. Auguste Lumiere, Andrée ...",A baby is seated at a table between its cheerf...,...,3693.0,NaN,NaN,5.9,NaN,NaN,NaN,NaN,NaN,NaN
tt0000070,Demolition of a Wall,1896,Not Rated,06 Mar 1896,1 min,"Documentary, Short",Louis Lumière,None,Auguste Lumière,Auguste Lumière directs four workers in the de...,...,2785.0,NaN,NaN,6.4,NaN,NaN,NaN,NaN,NaN,NaN


## EDA

In de dataset zijn er 18.060 films die bestaan het meeste uit kolommen met tekst zoals titel, genre, regisseur, plot en acteurs. Ook zijn er verschillende kolommen met metadata beschikbaar zoals jaar van release en de taal. Onderstaand is te zien dat er veel missende waarden zijn, waardoor het nodig is om dit op te schonen.

In [5]:
# overzicht van de data
df.shape
df.columns
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 18060 entries, tt0000010 to tt9916362
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Title                    18060 non-null  object 
 1   Year                     18060 non-null  object 
 2   Rated                    16024 non-null  object 
 3   Released                 18009 non-null  object 
 4   Runtime                  18050 non-null  object 
 5   Genre                    18058 non-null  object 
 6   Director                 18034 non-null  object 
 7   Writer                   17805 non-null  object 
 8   Actors                   18012 non-null  object 
 9   Plot                     18037 non-null  object 
 10  Language                 18010 non-null  object 
 11  Country                  18037 non-null  object 
 12  Awards                   13996 non-null  object 
 13  Poster                   18053 non-null  object 
 14  Metascore      

### Selectie van features
Om een content-based aanbeveler ga ik films vergelijken op basis van de inhoud. Daarom kies ik voor tekstuele en woorden die dezelfde betekenis hebben, synoniemen. 

- Plot: dit bevat de meeste informatie over de inhoud, zoals de verhaallijn.
- Genre: Laat de stijl zien van de films en zijn eenvoudig te vergelijken
- Actors en Directors: vaak zijn film geeks enthousiast over bepaalde actors of directors. Daarom is het belangrijk dat deze worden meegenomen in de vergelijking
- Year, language, country, type, imdbrating worden meegenomen als metadata om bijvoorbeeld de context be bepalen.

De rest neem ik niet mij, zoals posters en websites dragen niet bij aan de vergelijking tussen films.

In [6]:
#Selectie van relevante kolommen
columns_select = [
    "Title", "Year", "Genre", "Director", "Actors", "Plot",
    "Language", "Country", "Type", "imdbRating"
]

df_clean = df[columns_select].copy()
df_clean.head()

,Title,Year,Genre,Director,Actors,Plot,Language,Country,Type,imdbRating
imdbID,,,,,,,,,,
tt0000010,Leaving the Factory,1895,"Documentary, Short",Louis Lumière,None,A man opens the big gates to the Lumière facto...,None,France,movie,6.8
tt0000012,The Arrival of a Train,1896,"Documentary, Short","Auguste Lumière, Louis Lumière","Madeleine Koehler, Marcel Koehler, Mrs. August...",A group of people are standing in a straight l...,None,France,movie,7.4
tt0000014,The Waterer Watered,1895,"Short, Comedy",Louis Lumière,"François Clerc, Benoît Duval","A gardener is watering his flowers, when a mis...",None,France,movie,7.1
tt0000029,Baby's Meal,1895,"Documentary, Short",Louis Lumière,"Auguste Lumière, Mrs. Auguste Lumiere, Andrée ...",A baby is seated at a table between its cheerf...,None,France,movie,5.9
tt0000070,Demolition of a Wall,1896,"Documentary, Short",Louis Lumière,Auguste Lumière,Auguste Lumière directs four workers in the de...,None,France,movie,6.4


In [7]:
# Controle op missende waarden
df_clean.isna().sum()


Title          0
Year           0
Genre          2
Director      26
Actors        48
Plot          23
Language      50
Country       23
Type           0
imdbRating    29
dtype: int64

### Datarepresentatie

TF-IDF verwacht één tekstveld per item, ik ga alle tekst combineren tot één tekstveld. Dit zorgt ervoor dat de film meer informatie heeft dan alleen de plot.

Daarna ga ik een preprocessing toepassen, ik verander hoofdletters naar kleine letters, verwijderen van leestekens en het verwijderen van stopwoorden. Dit zorgt ervoor dat TF-IDF op andere termen gaan letten die van belang zijn.

In [8]:
# Tekstkolommen samenvoegen tot één kolom
text_cols = ["Genre", "Director", "Actors", "Plot", "Language", "Country"]

# Vul missende waarden in tekstkolommen met lege strings
df_clean[text_cols] = df_clean[text_cols].fillna("")

df_clean["text"] = (
    df_clean["Genre"] + " " +
    df_clean["Director"] + " " +
    df_clean["Actors"] + " " +
    df_clean["Plot"]
)
# Bekijk de eerste rij van de nieuwe tekstkolom
df_clean["text"].iloc[29]



'Short, Comedy, Drama Charles Chaplin Charles Chaplin, Edna Purviance, Dave Anderson Poor Charlie lives in a vacant lot. He tries to get a job but when he gets to the head of the employment line the jobs are gone. Back "home" he rescues Scraps, a bitch being attacked by other strays. Together they manage to steal some sausages from a lunch wagon. They enter a dance hall where Edna is a singer and unwilling companion to the clientele. He is thrown out when he can\'t pay. Back "home" Scraps digs up a money-filled wallet buried by crooks. They return to the dance hall to find Edna fired. The wallet goes back and forth between Charlie and the crooks. Charlie, Edna and Scraps end up very happily.'

In [9]:
# Statistieken over de lengte van de tekstkolom
df_clean["text"].str.len().describe()

count    18060.000000
mean       592.706202
std        365.588854
min          3.000000
25%        345.000000
50%        516.000000
75%        738.000000
max       5659.000000
Name: text, dtype: float64

In [10]:
# Tekstvoorbewerking functie
#def preprocess_text(text):

    # Omzetten naar kleine letters
    #text = text.lower()

    # Verwijderen van leestekens en speciale tekens
    #text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Verwijderen van stopwoorden
    #stop_words = set(stopwords.words("english"))
    #words = text.split()
    #words = [word for word in words if word not in stop_words]
    #return " ".join(words)


## Observatie speciale tekens

Namen met accenten zoals Méliès worden verwijderd naar mlis. De tekstrepresentatie wordt eenvoudigers maar het is ook een bias voor niet Engelstalige namen.

In [11]:
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # Zorg dat het altijd tekst is
    if not isinstance(text, str):
        return ""

    # Lowercase
    text = text.lower()

    # Normaliseer accenten: é → e, ö → o
    text = unicodedata.normalize("NFKD", text)
    text = "".join([c for c in text if not unicodedata.combining(c)])

    # Verwijder leestekens, maar behoud letters
    text = re.sub(r"[^a-z\s]", "", text)

    # Stopwoorden verwijderen
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return " ".join(words)


In [12]:
# Toepassen van de tekstvoorbewerking op de tekstkolom
df_clean["text_processed"] = df_clean["text"].apply(preprocess_text)

# Eerste twee rijen van de originele en verwerkte tekstkolom
df_clean[["text", "text_processed"]].head(29)


,text,text_processed
imdbID,,
tt0000010,"Documentary, Short Louis Lumière A man opens ...",documentary short louis lumiere man opens big ...
tt0000012,"Documentary, Short Auguste Lumière, Louis Lumi...",documentary short auguste lumiere louis lumier...
tt0000014,"Short, Comedy Louis Lumière François Clerc, Be...",short comedy louis lumiere francois clerc beno...
tt0000029,"Documentary, Short Louis Lumière Auguste Lumiè...",documentary short louis lumiere auguste lumier...
tt0000070,"Documentary, Short Louis Lumière Auguste Lumiè...",documentary short louis lumiere auguste lumier...
tt0000075,"Short, Horror Georges Méliès Jehanne d'Alcy, G...",short horror georges melies jehanne dalcy geor...
tt0000091,"Short, Horror Georges Méliès Jehanne d'Alcy, J...",short horror georges melies jehanne dalcy jule...
tt0000211,"Short, Comedy, Fantasy Georges Méliès Jehanne ...",short comedy fantasy georges melies jehanne da...
tt0000359,"Short, Comedy, Fantasy Georges Méliès Georges ...",short comedy fantasy georges melies georges me...


In [13]:
(df_clean["text_processed"].str.len() == 0).sum()


np.int64(1)

### Unsupervised methode: TF-IDF en cosine

De volgende stappen worden genomen om een unsupervised aanpak te gebruiken:
1. TF-IDF vectorisatie: hierdoor worden alle filmteksten omgezet in een numerieke vector, de woorden krijgen gewicht op basis van frequentie binnen een film en zeldzaamheid in de hele df.
2. Cosine-afstand: dit meet de fastand op basis van de inhoud, ipv lengte van de tekst.
3. NearestNeighbors: vindt voor een gekozen film de meest vergelijkbare items zonder volledige similarity-matrix te hoeven opslaan.

In [14]:
# TF-IDF vectorisatie

tfidf = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf.fit_transform(df_clean["text_processed"])

tfidf_matrix.shape


(18060, 5000)

In [15]:
# Experiment: verschillende aantallen TF–IDF-features
#for k in [2000, 3000, 5000]:
    #tfidf_tmp = TfidfVectorizer(max_features=k)
    #X_tmp = tfidf_tmp.fit_transform(df_clean["text_processed"])
    #print(k, X_tmp.shape)



In [16]:
nn = NearestNeighbors(metric="cosine", algorithm="brute")
nn.fit(tfidf_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [17]:
df_clean = df_clean.reset_index()
title_to_idx = pd.Series(df_clean.index, index=df_clean["Title"]).drop_duplicates()


In [18]:
def recommend(title, n=5):
    if title not in title_to_idx:
        return f"'{title}' niet gevonden in de dataset."

    idx = title_to_idx[title]
    distances, indices = nn.kneighbors(tfidf_matrix[idx], n_neighbors=n+1)

    # eerste resultaat is de film zelf dus deze overslaan
    rec_indices = indices.flatten()[1:]
    rec_distances = distances.flatten()[1:]

    out = df_clean.loc[rec_indices, ["Title", "Year", "Genre"]].copy()
    out["similarity"] = 1 - rec_distances  
    # cosine distance → similarity
    return out.reset_index(drop=True)


In [19]:
recommend("Runaway Bride", n=5)


,Title,Year,Genre,similarity
0,Herbie: Fully Loaded,2005,"Adventure, Comedy, Family",0.247777
1,Playdate with Destiny,2020,"Animation, Short, Comedy",0.245315
2,Shithouse,2020,"Comedy, Drama, Romance",0.243353
3,Instructions Not Included,2013,"Comedy, Drama",0.243234
4,Infinitely Polar Bear,2014,"Comedy, Drama, Romance",0.226008


In [20]:
recommend("Toy Story", n=5)


,Title,Year,Genre,similarity
0,Toy Story 4,2019,"Animation, Adventure, Comedy",0.434344
1,Toy Story 2,1999,"Animation, Adventure, Comedy",0.408488
2,Woody Allen: A Documentary,2011,"Documentary, Biography, History",0.334113
3,Lost in London,2017,"Comedy, Drama",0.329139
4,Toy Story 3,2010,"Animation, Adventure, Comedy",0.311049


De aanbevelingen laten zien dat het werkt voor duidelijke overeenkomsten, zoals toy story 4 en toy story 2. Deze horen namelijk tot dezelfde filmreeks. Alleen toy story 3 staat op plek 5, dit laat zien dat verschillen in tekst invloed hebben op de volgorde.

Ook komt Woody Allen: A Documentary terug, het systeem kijkt dus echt naar de woorden en niet naar de betekenis van de films.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def test_max_features(title, k, n=5):
    tfidf = TfidfVectorizer(max_features=k)
    X = tfidf.fit_transform(df_clean["text_processed"])

    nn = NearestNeighbors(metric="cosine", algorithm="brute")
    nn.fit(X)

    title_to_idx = pd.Series(df_clean.index, index=df_clean["Title"]).drop_duplicates()
    idx = title_to_idx[title]

    distances, indices = nn.kneighbors(X[idx], n_neighbors=n+1)
    rec_indices = indices.flatten()[1:]
    rec_distances = distances.flatten()[1:]

    out = df_clean.loc[rec_indices, ["Title", "Year", "Genre"]].copy()
    out["similarity"] = 1 - rec_distances
    out["max_features"] = k
    return out


### Iteratie k

Door de k aan te passen naar 7000 worden specifieke woorden meegenomen. Hierdoor worden films die uit dezelfde serie bestaan meegenomen. Dit zorgt ervoor dat toy story 3 hoger verschijnt. Maar dit zorgt er ook voor dat woorden worden meegenomen die zelden voorkomen. Hierdoor worden films toevallig met elkaar vergeleken en zo wordt de aanbevelingen beïnvloedt.

In [22]:
#test_max_features("Toy Story", k=7000)


# Reflectie

Deze aanbevelingssysteem laat zien dat het afhankt van de ontwerpkeuzes en dat kleine veranderingen zoals k 5000 of k 7000 invloed hebben op de uitkomsten.